# HW 3 - Synonym Detection In Spark
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__

In the last homework assignment you performed Naive Bayes to classify documents as 'ham' or 'spam.' In doing so, we relied on the implicit assumption that the list of words in a document can tell us something about the nature of that document's content. We'll rely on a similar intuition this week: the idea that, if we analyze a large enough corpus of text, the list of words that appear in small window before or after a vocabulary term can tell us something about that term's meaning.

This will be your first assignment working in Spark. You'll perform Synonym Detection by repurposing an algorithm commonly used in Natural Language Processing to perform document similarity analysis. In doing so you'll also become familiar with important datatypes for efficiently processing sparse vectors and a number of set similarity metrics (e.g. Cosine, Jaccard, Dice). By the end of this homework you should be able to:  
* ... __define__ the terms `one-hot encoding`, `co-occurrance matrix`, `stripe`, `inverted index`, `postings`, and `basis vocabulary` in the context of both synonym detection and document similarity analysis.
* ... __explain__ the reasoning behind using a word stripe to compare word meanings.
* ... __identify__ what makes set-similarity calculations computationally challenging.
* ... __implement__ stateless algorithms in Spark to build stripes, inverted index and compute similarity metrics.
* ... __apply__ appropriate metrics to assess the performance of your synonym detection algorithm. 


__`NOTE`__: your reading assignment for weeks 5 and 6 were fairly heavy and you may have glossed over the papers on dimension independent similarity metrics by [Zadeh et al](http://stanford.edu/~rezab/papers/disco.pdf) and pairwise document similarity by [Elsayed et al](https://terpconnect.umd.edu/~oard/pdf/acl08elsayed2.pdf). If you haven't already, this would be a good time to review those readings -- they are directly relevant to this assignment.

__Please refer to the `README` for homework submission instructions and additional resources.__

# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [3]:
import re
import ast
import time
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Run the next four cells to create your directory in dbfs
You do not need to understand this scala snippet. It simply dynamically fetches your user directory name so that any files you write can be saved in your own directory.

In [5]:
%scala
//*******************************************
// GET USERNAME AND USERHOME
//*******************************************
val tags = com.databricks.logging.AttributionContext.current.tags

// Get the user's name
val name = tags.getOrElse(com.databricks.logging.BaseTagDefinitions.TAG_USER, java.util.UUID.randomUUID.toString.replace("-", ""))
val username = if (name != "unknown") name else dbutils.widgets.get("databricksUsername")

val userhome = s"dbfs:/user/$username"

// println(userhome)

val userDataFrame = Seq(name,username,userhome)).toDF("name","username","userhome")
userDataFrame.createOrReplaceTempView( "userTable" )


tags: Map[com.databricks.logging.TagDefinition,String] = Map(TagDefinition(opId,) -> ShortLockHold-d03ed51cefed653f, TagDefinition(opTarget,) -> NotebookManager.runCommand, TagDefinition(clusterMemory,) -> 152576, TagDefinition(serverBackendName,Class name of the rpc server backend) -> com.databricks.backend.daemon.driver.DriverCorral, TagDefinition(notebookId,) -> 1660010934379910, TagDefinition(projectName,) -> auth, TagDefinition(eventWindowTime,) -> 8618538.639999926, TagDefinition(httpTarget,) -> /notebook/1660010934379910/command/1660010934379915, TagDefinition(buildHash,) -> "", TagDefinition(browserHash,) -> #notebook/1660010934379910/command/1660010934379917, TagDefinition(host,Host where the request is coming from.) -> 10.74.250.102, TagDefinition(notebookLanguage,) -> python, TagDefinition(sparkVersion,) -> 6.2.x-cpu-ml-scala2.11, TagDefinition(hostName,) -> cons-webapp-5, TagDefinition(httpMethod,) -> POST, TagDefinition(browserIdleTime,) -> 2543, TagDefinition(jettyRpcJettyVersion,) -> 8, TagDefinition(browserTabId,) -> c2edd676-42e4-4b75-8dd7-844e7610da1d, TagDefinition(sourceIpAddress,) -> 71.237.199.147, TagDefinition(browserUserAgent,) -> Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36, TagDefinition(orgId,) -> 7564214546094626, TagDefinition(userAgent,) -> Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36, TagDefinition(clusterId,) -> 0209-165713-prey714, TagDefinition(rootOpId,Unique identifier for a root of a trace (tree of ops).) -> 4505637675903098284_5050337239734148669_3c064dfe86674c2c95da84b1f271331f, TagDefinition(sessionId,) -> 36e2694deee36d386807145d3782e05e8c40b55a42f212a969cfa47f350636ce, TagDefinition(clusterCreator,) -> Webapp, TagDefinition(clientBranchName,) -> 3.12.1925, TagDefinition(clusterType,) -> spot, TagDefinition(browserHasFocus,) -> true, TagDefinition(userId,) -> 1524520769800548, TagDefinition(browserIsHidden,) -> false, TagDefinition(opType,) -> ShortLockHold, TagDefinition(sourcePortNumber,) -> 0, TagDefinition(user,) -> adam.sohn@ischool.berkeley.edu, TagDefinition(browserHostName,) -> dbc-b1c912e7-d804.cloud.databricks.com, TagDefinition(parentOpId,) -> ShortLockHold-d03ed51cefed64f3, TagDefinition(jettyRpcType,) -> InternalDriverBackendMessages$DriverBackendRequest)
name: String = adam.sohn@ischool.berkeley.edu
username: String = adam.sohn@ischool.berkeley.edu
userhome: String = dbfs:/user/adam.sohn@ischool.berkeley.edu
userDataFrame: org.apache.spark.sql.DataFrame = [name: string, username: string ... 1 more field]

In [6]:
# RUN THIS CELL AS IS
# This code snippet reads the user directory name, and stores is in a python variable.
# Next, it creates a folder inside your home folder, which you will use for files which you save inside this notebook.
name = 'adam.sohn@ischool.berkeley.edu'
username = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().apply('user')
userhome = 'dbfs:/user/' + username
#userhome = 'dbfs:/user/adam.sohn@ischool.berkeley.edu'
user_dict = [{'name':[name], 'username':[username], 'userhome':[userhome]}]
userDataFrame = spark.read.json(sc.parallelize(user_dict))
userDataFrame.createOrReplaceTempView( "userTable" )
userhome




Out[23]: 'dbfs:/user/adam.sohn@ischool.berkeley.edu'

In [7]:
# RUN THIS CELL AS IS. You should see multiple google-eng-all-5gram-* files in the results. If you do not see these, please let an Instructor or TA know.
display(dbutils.fs.ls('/mnt/mids-w261/data/HW3/'))

path,name,size
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-0-filtered.txt,googlebooks-eng-all-5gram-20090715-0-filtered.txt,11444614
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-1-filtered.txt,googlebooks-eng-all-5gram-20090715-1-filtered.txt,0
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-10-filtered.txt,googlebooks-eng-all-5gram-20090715-10-filtered.txt,11447003
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-100-filtered.txt,googlebooks-eng-all-5gram-20090715-100-filtered.txt,11484723
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-101-filtered.txt,googlebooks-eng-all-5gram-20090715-101-filtered.txt,11473190
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-102-filtered.txt,googlebooks-eng-all-5gram-20090715-102-filtered.txt,11411047
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-103-filtered.txt,googlebooks-eng-all-5gram-20090715-103-filtered.txt,11479296
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-104-filtered.txt,googlebooks-eng-all-5gram-20090715-104-filtered.txt,11426686
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-105-filtered.txt,googlebooks-eng-all-5gram-20090715-105-filtered.txt,11482267
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-106-filtered.txt,googlebooks-eng-all-5gram-20090715-106-filtered.txt,11466886


In [8]:
# RUN THIS CELL AS IS - Again, no need to worry about this, it simply creates your user-specific path variable in python
result = sqlContext.sql('select userhome from userTable')
userhome = result.first()[0]
hw3_path = 'dbfs:/user/adam.sohn@ischool.berkeley.edu/hw3/'#userhome + "/hw3/" 
hw3_path_open = '/dbfs' + hw3_path.split(':')[-1] # for use with python open()
dbutils.fs.mkdirs(hw3_path)

Out[37]: True

In [9]:
# RUN THIS CELL AS IS - A test to make sure your directory is working as expected.
# You should see a result like:
# dbfs:/user/youremail@ischool.berkeley.edu/hw3/sample_docs.txt
dbutils.fs.put(hw3_path+'test.txt',"hello world",True)
display(dbutils.fs.ls(hw3_path))

path,name,size
dbfs:/user/adam.sohn@ischool.berkeley.edu/demo4/,demo4/,0
dbfs:/user/adam.sohn@ischool.berkeley.edu/demo6/,demo6/,0
dbfs:/user/adam.sohn@ischool.berkeley.edu/hw3/,hw3/,0


In [10]:
# get Spark Session info (RUN THIS CELL AS IS)
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 spark://10.74.224.148:7077 
 AppName 
 Databricks Shell

In [11]:
# start SparkContext (RUN THIS CELL AS IS)
sc = spark.sparkContext

In [12]:
# OPTIONAL
# Spark configuration Information (RUN THIS CELL AS IS)
# sc.getConf().getAll()

__`REMINDER:`__ If you are running this notebook in databricks, you can monitor the progress of your jobs using the Spark UI by clicking on "view" in the output cell below the cell you are running

# Question 1: Spark Basics.
In your readings and live session demos for weeks 4 and 5 you got a crash course in working with Spark. We also talked about how Spark RDDs fit into the broader picture of distributed algorithm design. The questions below cover key points from these discussions. Feel free to answer each one very briefly.

### Q1 Tasks:

* __a) short response:__ What is Spark? How  does it relate to Hadoop MapReduce?

* __b) short response:__ In what ways does Spark follow the principles of statelessness (a.k.a. functional programming)? List at least one way in which it allows the programmer to depart from this principle. 

* __c) short response:__ In the context of Spark what is a 'DAG' and how do they relate to the difference between an 'action' and a 'transformation'? Why is it useful to pay attention to the DAG that underlies your Spark implementation?

* __d) short response:__ Give a specific example of when we would want to `cache()` an RDD and explain why.

### Q1 Student Answers:
> __a)__ <br>
What is Spark - (referencing High Performance Spark)<br>Apache Spark is an open source framework that provides methods to process data in parallel that are generalizable; the same high-level Spark functions can be used to perform disparate data processing tasks on data of different sizes and structures. <br><br>
How does Spark relate to MapReduce - (referencing Async 4.2) Spark is compatible w/ an hdfs file system, just as MapReduce leverages. However, there are differences, with most of which being relative advantages for Spark. The Spark pipeline is able to better leverage memory vs. write-to-disk, which improves speed (Spark is 100x faster than MapReduce). Also, Spark offers added transparancy into its operation by read-eval-print loop (REPL). Hadoop does not have REPL. There are more advantages, such as High-level data management, fault tolerance, first class framework for distributing programming over a large volume of data.<br><br>

> __b)__ <br>
(referencing 'Learning Spark')<br>
Spark/Stateless - An example of Spark following the principle of statelessness is that transformations of invidividual records (for example, in an RDD) do not depend on other records.<br><br>
Departure from Stateless - Spark has some functions which are intentionally stateful. For example, updateStateByKey() tracks the state of a key as it changes over record events.<br><br>

> __c)__ <br>
(referencing 'High Performance Spark')<br>
What is DAG - Spark’s high-level scheduling layer uses RDD dependencies to build a Directed Acyclic Graph (a DAG) of stages for each Spark job.
How does DAG relate to action/transformation - Actions trigger the scheduler, which builds the DAG, based on the dependencies between RDD transformations. In other words, Spark evaluates an action by working backward to define the series of steps it has to take to produce each object in the final distributed dataset (each partition). Then, using this series of steps, called the execution plan, the scheduler computes the missing partitions for each stage until it computes the result.<br><br>
Why pay attention to DAG - The DAG can be viewed by the data scientist, which helps offer a Gross Reality Check on the built pipeline functionalty.<br><br>

> __d)__ For example, if you were performing a loop of joins to the same dataset, persisting (with cache() as an option) that dataset could lead to huge performance improvements since it ensures that the partitions of that RDD will be available in-memory (as opposed to re-computing from disk )to do each join.

# Question 2: Similarity Metrics
As mentioned in the introduction to this assignment, an intuitive way to compare the meaning of two documents is to compare the list of words they contain. Given a vocabulary $V$ (feature set) we would represent each document as a vector of `1`-s and `0`-s based on whether or not it contains each word in $V$. These "one-hot encoded" vector representations allow us to use math to identify similar documents. However like many NLP tasks the high-dimensionality of the feature space is a challenge... especially when we start to scale up the size and number of documents we want to compare.

In this question we'll look at a toy example of document similarity analysis. Consider these 3 'documents': 
```
docA	the flight of a bumblebee
docB	the length of a flight
docC	buzzing bumblebee flight
```
These documents have a total of $7$ unique words: 
>`a, bumblebee, buzzing, flight, length, of, the`.     

Given this vocabulary, the documents' vector representations are (note that one-hot encoded entries follow the order of the vocab list above):

```
docA	[1,1,0,1,0,1,1]
docB	[1,0,0,1,1,1,1]
docC	[0,1,1,1,0,0,0]
```  

### Q2 Tasks:

* __a) short response:__ Explain what the the numerator and denominator of this calculation would represent in terms of word counts in documents A and B.

* __b) short response:__ Explain how the Jaccard, Overlap and Dice metrics are similar/different to the calculation for cosine similarity. When would these metrics lead to different similarity rankings for a set of documents?

* __c) short response:__ Calculate the cosine similarity for each pair of documents in our toy corpus. Please use markdown and $\LaTeX$ to show your calcuations.  

* __d) short response:__ According to your calculations in `part c` which pair of documents are most similar in meaning? Does this match your expecatation from reading the documents? If not, speculate about why we might have gotten this result.

* __e) short response:__ In NLP common words like '`the`', '`of`', and '`a`' increase our feature space without adding a lot of signal about _semantic meaning_. Repeat your analysis from `part c` but this time ignore these three words in your calculations [__`TIP:`__ _to 'remove' stopwords just ignore the vector entries in columns corresponding to the words you wish to disregard_]. How do your results change?

### Q2 Student Answers:
> __a)__ :<br>
(source: http://stanford.edu/~rezab/papers/disco.pdf)<br>
As compared to word counts of docs A,B:<br>
the numerator of the cosine similarity has a minimum value of 0 (no overlapping terms) and a maximum value of 7 (each term in at least 1 doc matched in other doc)<br>
the denominator of the cosine similarity has a value of sqrt(word_count_docA) * sqrt(word_count_docB), which must be a positive integer.<br><br>

> __b)__ <br>
(source: http://stanford.edu/~rezab/papers/disco.pdf)<br>
Comparison - A comparison of the calculations is shown below:
![Cosine_Jaccard_Overlap_Dice]<br>
Different rankings - Cosine, Jaccard, Overlap and Dice metrics are expected to differ based on inputs, as they are different functions. A comparison of bookend cases is below. Interestingly, for the data set chosen, Overlap cited a 'some overlap' case as similarity ranking of 1. <br>
docW = \[1,1,0,0]<br>
docX = \[1,0,0,0]<br>
docY = \[0,0,1,1]<br>
docZ = \[1,1,0,0]<br>

|   Scenario                | Cosine | Jaccard | Overlap | Dice |
|---------------------------|--------|---------|---------|------|
| No overlap (docW,docY)    |   0.0  |   0.0   |   0.0   | 0.0  |
| Some overlap (docW, docX) |   0.7  |   0.5   |   1.0   | 0.7  |
| Full overlap (docW,docZ)  |   1.0  |   1.0   |   1.0   | 1.0  |

> __c)__ <br>
$sim(A,B) = \frac{A^T\cdot B}{\sqrt{\sum A}\cdot \sqrt{\sum B}}$<br>
$sim(A,B) = \frac{1\cdot 1 + 1\cdot 0 + 0\cdot 0 + 1\cdot 1 + 0\cdot 1 + 1\cdot 1 + 1\cdot 1}{\sqrt{1+1+0+1+0+1+1}\cdot \sqrt{1+0+0+1+1+1+1}}$<br>
$sim(A,B) = 0.8$<br><br>
$sim(A,C) = \frac{A^T\cdot C}{\sqrt{\sum A}\cdot \sqrt{\sum C}}$<br>
$sim(A,C) = \frac{1\cdot 0 + 1\cdot 1 + 0\cdot 1 + 1\cdot 1 + 0\cdot 0 + 1\cdot 0 + 1\cdot 0}{\sqrt{1+1+0+1+0+1+1}\cdot \sqrt{0+1+1+1+0+0+0}}$<br>
$sim(A,C) = 0.5$<br><br>
$sim(B,C) = \frac{B^T\cdot C}{\sqrt{\sum B}\cdot \sqrt{\sum C}}$<br>
$sim(B,C) = \frac{1\cdot 0 + 0\cdot 1 + 0\cdot 1 + 1\cdot 1 + 1\cdot 0 + 1\cdot 0 + 1\cdot 0}{\sqrt{1+0+0+1+1+1+1}\cdot \sqrt{0+1+1+1+0+0+0}}$<br>
$sim(B,C) = 0.3$<br><br>

> __d)__ According to calculations above, cosine_sim(A,B) is the greatest of the document set at 0.8. From a human's reading of the documents, one would argue that docs A,C are the most similar, as these docs have 2 subjects (bumblebee, flight) matched, whereas A,B & B,C only match on a single subject (flight). The reason the algorithm chose highest similarity for A,B because of the many articles ('the','of','a') shared between A,B. 

> __e)__ Rerunning the analysis while excluding articles ('the','of','a'), documents A,C now have the highest cosine similarity at 0.8.<br><br>

$sim(A,B) = \frac{A^T\cdot B}{\sqrt{\sum A}\cdot \sqrt{\sum B}}$<br>
$sim(A,B) = \frac{1\cdot 0 + 0\cdot 0 + 1\cdot 1 + 0\cdot 1}{\sqrt{1+0+1+0}\cdot \sqrt{0+0+1+1}}$<br>
$sim(A,B) = $0.5<br><br>
$sim(A,C) = \frac{A^T\cdot C}{\sqrt{\sum A}\cdot \sqrt{\sum C}}$<br>
$sim(A,C) = \frac{1\cdot 1 + 0\cdot 1 + 1\cdot 1 + 0\cdot 0 }{\sqrt{1+0+1+0}\cdot \sqrt{1+1+1+0}}$<br>
$sim(A,C) = $0.8<br><br>
$sim(B,C) = \frac{B^T\cdot C}{\sqrt{\sum B}\cdot \sqrt{\sum C}}$<br>
$sim(B,C) = \frac{0\cdot 1 + 0\cdot 1 + 1\cdot 1 + 1\cdot 0 }{\sqrt{0+0+1+1}\cdot \sqrt{1+1+1+0}}$<br>
$sim(B,C) = $0.4<br><br>

# Question 3: Synonym Detection Strategy

In the Synonym Detection task we want to compare the meaning of words, not documents. For clarity, lets call the words whose meaning we want to compare `terms`. If only we had a 'meaning document' for each `term` then we could easily use the document similarity strategy from Question 2 to figure out which `terms` have similar meaning (i.e. are 'synonyms'). Of course in order for that to work we'd have to reasonably believe that the words in these 'meaning documents' really do reflect the meaning of the `term`. For a good analysis we'd also need these 'meaning documents' to be fairly long -- the one or two sentence dictionary definition of a term isn't going to provide enough signal to distinguish between thousands and thousands of `term` meanings.

This is where the idea of co-occurrance comes in. Just like DocSim makes the assumption that words in a document tell us about the document's meaning, we're going to assume that the set of words that 'co-occur' within a small window around our term can tell us some thing about the meaning of that `term`. Remember that we're going to make this 'co-words' list (a.k.a. 'stripe') by looking at a large body of text. This stripe is our 'meaning document' in that it reflects all the kinds of situations in which our `term` gets used in real language. So another way to phrase our assumption is: we think `terms` that get used to complete lots of the same phrases probably have related meanings. This may seem like an odd assumption but computational linguists have found that it works surprisingly well in practice. Let's look at a toy example to build your intuition for why and how.

Consider the opening line of Charles Dickens' _A Tale of Two Cities_:

In [19]:
corpus = """It was the best of times, it was the worst of times, 
it was the age of wisdom it was the age of foolishness"""

There are a total of 10 unique words in this short 'corpus':

In [21]:
words = list(set(re.findall(r'\w+', corpus.lower())))
print(words)

['was', 'age', 'times', 'foolishness', 'the', 'worst', 'of', 'wisdom', 'it', 'best']

But of these 10 words, 4 are so common that they probably don't tell us very much about meaning.

In [23]:
stopwords = ["it", "the", "was", "of"]

So we'll ignore these 'stop words' and we're left with a 6 word vocabulary:

In [25]:
vocab = sorted([w for w in words if w not in stopwords])
print(vocab)

['age', 'best', 'foolishness', 'times', 'wisdom', 'worst']

Your goal in the tasks below is to asses, which of these six words are most related to each other in meaning -- based solely on this short two line body of text.

### Q3 Tasks:

* __a) short response:__ Given this six word vocabulary, how many 'pairs' of words do we want to compare? More generally for a n-word vocabulary how many pairwise comparisons are there to make? 

* __b) code:__ In the space provided below, create a 'stripe' for each `term` in the vocabulary. This stripe should be the list of all other vocabulary words that occur within a __5 word window__ (two words on either side) of the `term`'s position in the original text.

* __c) code + short response:__ Complete the provided code to turn your stripes into a 1-hot encoded co-occurrence matrix. For our 6 word vocabulary how many entries are in this matrix? How many entries are zeros? 

* __d) code:__ Complete the provided code to loop over all pairs and compute their cosine similarity. Please do not modify the existing code, just add your own in the spot marked.

* __e) short response:__ Which pairs of words have the highest 'similarity' scores? Are these words 'synonyms' in the traditional sense? In what sense are their meanings 'similar'? Explain how our results are contingent on the input text. What would change if we had a much larger corpus?

### Q3 Student Answers:
> __a)__ Assuming we are excluding self-self pairs, there are $\frac{N\cdot (N+1)}{2}$ combinations of pairs in a list. For N=6, this means 21 combinations. <br>
<br>
> __c)__ There are 36 entries in a co-occurrence matrix for a 6-word vocabulary (including self-self). Of the 36 entries, 28 are 0's, 8 are 1's. The reason the co-occurence matrix entry count is higher than the unique pair count is that co-occurence matrix includes self-self combinations and counts each self-other combination twice. <br><br>

> __e)__ <br>
Which highest similarity score:  The highest similarity scores are for 'foolishness-wisdom' and 'best-worst'. <br>
Synonyms -  'foolishness-wisdom' and 'best-worst' are NOT synonyms. They actually have opposite meanings.<br>
How Similar - 'foolishness-wisdom' and 'best-worst' are similar in that they are both valid treatments in an otherwise identical sentence. That is, they could be swapped out and still have a valid sentence.<br>
How results contingent on input - Our similarity algorithm takes input texts, and only input texts, as its input. As words show up in multiple texts in similar contexts (as demonstrated by stripe), they are identified as similar. <br>
What if larger corpus - A larger corpus provides more opportunities for more words to show up in texts. This larger data set would likely show more subtle relationships (as opposed to not recognizing them) and not over-represent subtle relationships.

In [28]:
# for convenience, here are the corpus & vocab list again (RUN THIS CELL AS IS)
print("CORPUS:")
print(corpus)
print('VOCAB:')
print(vocab)

CORPUS:
It was the best of times, it was the worst of times, 
it was the age of wisdom it was the age of foolishness
VOCAB:
['age', 'best', 'foolishness', 'times', 'wisdom', 'worst']

<img src="https://raw.githubusercontent.com/kyleiwaniec/MIDS_CV/gh-pages/best-of-times.png" />

In [30]:
# part b - USE THE TEXT ABOVE TO COMPLETE EACH STRIPE
stripes = {'age':['wisdom','foolishness'], # example
           'best':['times'], # YOU FILL IN THE REST
           'foolishness':['age'],
           'times': ['best', 'worst'],
           'wisdom':['age'],
           'worst':['times']}

In [31]:
# part c - initializing an empty co-occurrence matrix (RUN THIS CELL AS IS)
co_matrix = pd.DataFrame({term: [0]*len(vocab) for term in vocab}, index = vocab, dtype=int)

In [32]:
# part c - FILL IN THE MISSING LINE so that this cell 1-hot encodes the co-occurrence matrix
for term, nbrs in stripes.items():
    for nbr in nbrs:
        pass
        ############# YOUR CODE HERE #################
        co_matrix[term][vocab.index(nbr)] = 1
        co_matrix[nbr][vocab.index(term)] = 1
        ############# (END) YOUR CODE #################
co_matrix

,age,best,foolishness,times,wisdom,worst
age,0,0,1,0,1,0
best,0,0,0,1,0,0
foolishness,1,0,0,0,0,0
times,0,1,0,0,0,1
wisdom,1,0,0,0,0,0
worst,0,0,0,1,0,0


In [33]:
# part e - FILL IN THE MISSING LINES to compute the cosine similarity between each pair of terms
for term1, term2 in itertools.combinations(vocab, 2):
    # one hot-encoded vectors
    v1 = co_matrix[term1]
    v2 = co_matrix[term2]
    
    # cosine similarity
    ############# YOUR CODE HERE #################
    csim = None
    sum_sqrt_v1 = sum([i**0.5 for i in v1])
    sum_sqrt_v2 = sum([i**0.5 for i in v2])
    csim = sum(v1 * v2)/(sum_sqrt_v1 * sum_sqrt_v2)
    ############# (END) YOUR CODE #################    
    
    print(f"{term1}-{term2}: {csim}")

age-best: 0.0
age-foolishness: 0.0
age-times: 0.16666666666666666
age-wisdom: 0.0
age-worst: 0.0
best-foolishness: 0.0
best-times: 0.0
best-wisdom: 0.0
best-worst: 0.5
foolishness-times: 0.0
foolishness-wisdom: 1.0
foolishness-worst: 0.5
times-wisdom: 0.0
times-worst: 0.0
wisdom-worst: 0.5

# Question 4: Pairs and Stripes at Scale

As you read in the paper by Zadeh et al, the advantage of metrics like Cosine, Dice, Overlap and Jaccard is that they are dimension independent -- that is to say, if we implement them in a smart way the computational complexity of performing these computations is independent of the number of documents we want to compare (or in our case, the number of terms that are potential synonyms). One component of a 'smart implementation' involves thinking carefully both about how you define the "basis vocabulary" that forms your feature set (removing stopwords, etc). Another key idea is to use a data structure that facilitates distributed calculations. The DISCO implemetation further uses a sampling strategy, but that is beyond the scope of this assignment. 

In this question we'll take a closer look at the computational complexity of the synonym detection approach we took in question 3 and then revist the document similarity example as a way to explore a more efficient approach to parallelizing this analysis.

### Q4 Tasks:

* __a) short response:__ In question 3 you calculated the cosine similarity of pairs of words using the vector representation of their co-occurrences in a corpus. Imagine for now that you have unlimited memory on each of your nodes and describe a sequence of map & reduce steps that would start from a raw corpus and reproduce your strategy from Q3. For each step be sure to note what information would be stored in memory on your nodes and what information would need to be shuffled over the network (a bulleted list of steps with 1-2 sentences each is sufficient to answer this question).

* __b) short response:__ In the asynch videos about "Pairs and Stripes" you were introduced to an alternative strategy. Explain two ways that using these data structures are more efficient than 1-hot encoded vectors when it comes to distributed similarity calculations [__`HINT:`__ _Consider memory constraints, amount of information being shuffled, amount of information being transfered over the network, and level of parallelization._]

* __c) read provided code:__ The code below provides a streamined implementation of Document similarity analysis in Spark. Read through this code carefully. Once you are confident you understand how it works, answer the remaining questions. [__`TIP:`__ _to see the output of each transformation try commenting out the subsequent lines and adding an early `collect()` action_.]

* __d) short response:__ The second mapper function, `splitWords`, emits 'postings'. The list of all 'postings' for a word is also refered to as an 'inverted index'. In your own words, define each of these terms ('postings' and 'inverted index') based on your reading of the provided code. (*DITP by Lin and Dyer also contains a chapter on the Inverted Index although in the context of Hadoop rather than Spark*).

* __e) short response:__ The third mapper, `makeCompositeKeys`, loops over the inverted index to emit 'pairs' of what? Explain what information is included in the composite key created at this stage and why it makes sense to synchronize around that information in the context of performing document similarity calculations. In addition to the information included in these new keys, what other piece of information will we need to compute Jaccard or Cosine similarity?

* __f) short response:__ Out of all the Spark transformations we make in this analysis, which are 'wide' transformations and which are 'narrow' transformations. Explain.

### Q4 Student Answers:
> __a)__ <br>
Assume we start w/ an understanding of stopwords

* MAP - From corpus (read from disk into memory), filter stopwords & emit (in memory) word pairs in form ((term1,term2),1) where (term1,term2) is key and 1 is value.
* COMBINE - Given the approach of one-hot encoding the term vectors and on-hot encoding the co-occurence matrix, the combiner would not seek to sum values of like keys. The combiner would instead ensure that there is only a single entry (if any) for each ordered key pair. These single entries (if any) per key would then be emited over network.
* SHUFFLE - Shuffle would take in data from network, and in-memory, assign keys to Reducers according to default logic. Data would then output over network.
* REDUCE - Much scope in reducer!
  *  Calculate co-matrix. As the co-matrix is completed, the results are held in memory.
  *  From co-matrix, for each term vector, sqrt(sum values) for a component of cosine denomenator. Hold result in memory.
  *  From co-matrix, for each term vector, calculate the sum-product of each term vector combination. Hold result in memory.
  *  Combine elements from steps 2,3 above according to cosine similarity algorithm to emit ((term1, term2), cosine similarity) to disk.

> __b)__ Using a dictionary data structure emitted from the mapper in the alternative stripe method, the the alternative stripe method emits fewer, denser, packets, which (1 of 2 ways:) reduces the sorting/shuffler load and (2 of 2 ways:) allows Spark to perform combining behind the scenes from our code. 

> __c)__ _read provided code before answering d-f_ 

> __d)__<br> Postings are collections of metadata regarding the occurence of terms in documents. In the case of Mapper2, postings are comprised of `doc ID containing word`, `word count for doc ID containing word`. <br>Inverted Index - Given an input term, an Inverted Index is an array of postings where the term is present. Search engines leveraging Inverted Index can provide access to the list of documents (web pages, PDFs, code, etc) that contain the term according to a ranking model. <br><br>

> __e)__ Mapper3 emits all pairs of postings, with a counter `1`. As defined by Mapper2, posting is (document ID, word count per document ID). This information is important, as this is the information needed for all further similarity calculations. Otherwise, we will need the total count of composite keys.


> __f)__ Narrow transformation is where each input partition results in a single output partition. A wide transformation is where each input partition results in more than a single output partition. A wide transformation is also referred to as a shuffle. <br>
result = data.map(lambda line: line.split('\t')) \ __narrow__<br>
             .flatMap(splitWords) \ __narrow__<br>
             .reduceByKey(lambda x,y : x+y) \ __wide__ <br>
             .flatMap(makeCompositeKey) \ __narrow__<br>
             .reduceByKey(lambda x,y : x+y) \ __wide__ <br>
             .flatMap(jaccard) \ __narrow__<br>
             .takeOrdered(10, key=lambda x: -x[1]) __not a transform__ <br>

A small test file: __`sample_docs.txt`__

In [37]:
# RUN THIS CELL AS IS
dbutils.fs.put(hw3_path+"sample_docs.txt", 
"""docA	bright blue butterfly forget
docB	best forget bright sky
docC	blue sky bright sun
docD	under butterfly sky hangs
docE	forget blue butterfly""", True)

Wrote 144 bytes.
Out[7]: True

In [38]:
# RUN THIS CELL AS IS
print(dbutils.fs.head(hw3_path+"sample_docs.txt"))

docA	bright blue butterfly forget
docB	best forget bright sky
docC	blue sky bright sun
docD	under butterfly sky hangs
docE	forget blue butterfly

__Document Similarity Analysis in Spark:__

In [40]:
# load data - RUN THIS CELL AS IS
data = sc.textFile(hw3_path+"sample_docs.txt")  

In [41]:
# helper function - RUN THIS CELL AS IS
def splitWords(pair):
    """Mapper 2: tokenize each document and emit postings."""
    doc, text = pair
    words = text.split(" ")
    for w in words:
        yield (w, [(doc,len(words))])

In [42]:
# helper function - RUN THIS CELL AS IS
def makeCompositeKey(inverted_index):
    """Mapper 3: loop over postings and yield pairs."""
    word, postings = inverted_index
    # taking advantage of symmetry, output only (a,b), but not (b,a)
    for subset in itertools.combinations(sorted(postings), 2):
        yield (str(subset), 1)

In [43]:
# helper function - RUN THIS CELL AS IS
def jaccard(line):
    """Mapper 4: compute similarity scores"""
    (doc1, n1), (doc2, n2) = ast.literal_eval(line[0])
    total = int(line[1])
    jaccard = total / float(int(n1) + int(n2) - total)
    yield doc1+" - "+doc2, jaccard

In [44]:
# Spark Job - RUN THIS CELL AS IS
result = data.map(lambda line: line.split('\t')) \
             .flatMap(splitWords) \
             .reduceByKey(lambda x,y : x+y) \
             .flatMap(makeCompositeKey) \
             .reduceByKey(lambda x,y : x+y) \
             .flatMap(jaccard) \
             .takeOrdered(10, key=lambda x: -x[1])
result

Out[13]: [('docA - docE', 0.75),
 ('docA - docB', 0.3333333333333333),
 ('docA - docC', 0.3333333333333333),
 ('docB - docC', 0.3333333333333333),
 ('docD - docE', 0.16666666666666666),
 ('docB - docE', 0.16666666666666666),
 ('docC - docE', 0.16666666666666666),
 ('docC - docD', 0.14285714285714285),
 ('docA - docD', 0.14285714285714285),
 ('docB - docD', 0.14285714285714285)]

# About the Data
Now that you are comfortable with similarity metrics we turn to the main task in this assignment: Synonym Detection. As you saw in Question 3 the ability of our algorithm to detect words with similar meanings is highly dependent on our input text. Specifically, we need a large enough corpus of natural language that we can expose our algorithm to a realistic range of contexts in which any given word might get used. Ideally, these 'contexts' would also provide enough signal to distinguish between words with similar semantic roles but different meaning. Finding such a corpus will be easier to accomplish for some words than others.

For the main task in this portion of the homework you will use data from Google's n-gram corpus. This data is particularly convenient for our task because Google has already done the first step for us: they windowed over a large subset of the web and extracted all 5-grams. If you are interested in learning more about this dataset the original source is: http://books.google.com/ngrams/, and a large subset is available [here from AWS](https://aws.amazon.com/datasets/google-books-ngrams/). 

For this assignment we have provided a subset of the 5-grams data consisting of 191 files of approximately 10MB each. These files are available in dbfs. Please only use the provided data so that we can ensure consistent results from student to student.

Each row in our dataset represents one of these 5 grams in the format:
> `(ngram) \t (count) \t (pages_count) \t (books_count)`

__DISCLAIMER__: In real life, we would calculate the stripes cooccurrence data from the raw text by windowing over the raw text and not from the 5-gram preprocessed data.  Calculating pairs on this 5-gram is a little corrupt as we will be double counting cooccurences. Having said that this exercise can still pull out some similar terms.

In [46]:
# RUN THIS CELL AS IS. You should see multiple google-eng-all-5gram-* files in the results. If you do not see these, please let an Instructor or TA know.
display(dbutils.fs.ls('/mnt/mids-w261/data/HW3/'))

path,name,size
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-0-filtered.txt,googlebooks-eng-all-5gram-20090715-0-filtered.txt,11444614
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-1-filtered.txt,googlebooks-eng-all-5gram-20090715-1-filtered.txt,0
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-10-filtered.txt,googlebooks-eng-all-5gram-20090715-10-filtered.txt,11447003
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-100-filtered.txt,googlebooks-eng-all-5gram-20090715-100-filtered.txt,11484723
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-101-filtered.txt,googlebooks-eng-all-5gram-20090715-101-filtered.txt,11473190
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-102-filtered.txt,googlebooks-eng-all-5gram-20090715-102-filtered.txt,11411047
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-103-filtered.txt,googlebooks-eng-all-5gram-20090715-103-filtered.txt,11479296
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-104-filtered.txt,googlebooks-eng-all-5gram-20090715-104-filtered.txt,11426686
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-105-filtered.txt,googlebooks-eng-all-5gram-20090715-105-filtered.txt,11482267
dbfs:/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-106-filtered.txt,googlebooks-eng-all-5gram-20090715-106-filtered.txt,11466886


In [47]:
# set global paths to full data folder and to the first file (which we'll use for testing)
NGRAMS = '/mnt/mids-w261/data/HW3'
F1_PATH = '/mnt/mids-w261/data/HW3/googlebooks-eng-all-5gram-20090715-0-filtered.txt'

As you develop your code you should use the following file to systems test each of your solutions before running it on the Google data. (Note: these are the 5-grams extracted from our two line Dickens corpus in Question 3... you should find that your Spark job results match the calculations we did "by hand").

Test file: __`systems_test.txt`__

In [49]:
# RUN THIS CELL AS IS
dbutils.fs.put(hw3_path+"systems_test.txt",
"""it was the best of	1	1	1
age of wisdom it was	1	1	1
best of times it was	1	1	1
it was the age of	2	1	1
it was the worst of	1	1	1
of times it was the	2	1	1
of wisdom it was the	1	1	1
the age of wisdom it	1	1	1
the best of times it	1	1	1
the worst of times it	1	1	1
times it was the age	1	1	1
times it was the worst	1	1	1
was the age of wisdom	1	1	1
was the best of times	1	1	1
was the age of foolishness	1	1	1
was the worst of times	1	1	1
wisdom it was the age	1	1	1
worst of times it was	1	1	1""",True)

Wrote 493 bytes.
Out[16]: True

Finally, we'll create a Spark RDD for each of these files so that they're easy to access throughout the rest of the assignment.

In [51]:
# RUN THIS CELL AS IS Spark RDDs for each dataset
testRDD = sc.textFile(hw3_path+"systems_test.txt") 
f1RDD = sc.textFile(F1_PATH)
dataRDD = sc.textFile(NGRAMS)

Let's take a peak at what each of these RDDs looks like:

In [53]:
testRDD.take(10)

Out[39]: ['it was the best of\t1\t1\t1',
 'age of wisdom it was\t1\t1\t1',
 'best of times it was\t1\t1\t1',
 'it was the age of\t2\t1\t1',
 'it was the worst of\t1\t1\t1',
 'of times it was the\t2\t1\t1',
 'of wisdom it was the\t1\t1\t1',
 'the age of wisdom it\t1\t1\t1',
 'the best of times it\t1\t1\t1',
 'the worst of times it\t1\t1\t1']

In [54]:
f1RDD.take(10)

Out[36]: ['A BILL FOR ESTABLISHING RELIGIOUS\t59\t59\t54',
 'A Biography of General George\t92\t90\t74',
 'A Case Study in Government\t102\t102\t78',
 'A Case Study of Female\t447\t447\t327',
 'A Case Study of Limited\t55\t55\t43',
 "A Child's Christmas in Wales\t1099\t1061\t866",
 'A Circumstantial Narrative of the\t62\t62\t50',
 'A City by the Sea\t62\t60\t49',
 'A Collection of Fairy Tales\t123\t117\t80',
 'A Collection of Forms of\t116\t103\t82']

In [55]:
dataRDD.take(10)

Out[34]: ['A BILL FOR ESTABLISHING RELIGIOUS\t59\t59\t54',
 'A Biography of General George\t92\t90\t74',
 'A Case Study in Government\t102\t102\t78',
 'A Case Study of Female\t447\t447\t327',
 'A Case Study of Limited\t55\t55\t43',
 "A Child's Christmas in Wales\t1099\t1061\t866",
 'A Circumstantial Narrative of the\t62\t62\t50',
 'A City by the Sea\t62\t60\t49',
 'A Collection of Fairy Tales\t123\t117\t80',
 'A Collection of Forms of\t116\t103\t82']

# Question 5: N-gram EDA part 1 (words)

Before starting our synonym-detection, let's get a sense for this data. As you saw in questions 3 and 4 the size of the vocabulary will impact the amount of computation we have to do. Write a Spark job that will accomplish the three tasks below as efficiently as possible. (No credit will be awarded for jobs that sort or subset after calling `collect()`-- use the framework to get the minimum information requested). As you develop your code, systems test each job on the provided file with Dickens ngrams, then on a single file from the Ngram dataset before running the full analysis.


### Q5 Tasks:
* __a) code:__ Write a Spark application to retrieve:
  * The number of unique words that appear in the data. (i.e. size of the vocabulary) 
  * A list of the top 10 words & their counts.
  * A list of the bottom 10 words & their counts.  
  
  __`NOTE  1:`__ _don't forget to lower case the ngrams before extracting words._  
  __`NOTE  2:`__ _don't forget to take in to account the number of occurances of each ngram._  
  __`NOTE  3:`__ _to make this code more reusable, the `EDA1` function code base uses a parameter 'n' to specify the number of top/bottom words to print (in this case we've requested 10)._


* __b) short response:__ Given the vocab size you found in part a, how many potential synonym pairs could we form from this corpus? If each term's stripe were 1000 words long, how many individual 'postings' tuples would we need to shuffle inorder to form the inverted indices? Show and briefly explain your calculations for each part of this question. [__`HINT:`__ see your work from q4 for a review of these concepts.]

* __c) short response:__ What do you notice about the most frequent words, how usefull will these top words be in synonym detection? Explain.

* __d) short response:__ What do you notice/infer about the least frequent words, how reliable should we expect the detected 'synonyms' for the bottom words to be? Explain.

### Q5 Student Answers:

> __b)__ <br>
Synonym Pairs - With a vocab size of `269339` for `dataRDD`, the highest potentional synonym pairs is equivalent to the number of distinct pairs. Mathematically, the number of distinct pairs is the Combination of `269339`, choose 2.  Calculation below:<br>
$_nC_r = \frac{n!}{r!\cdot(n-r)!} = \frac{269339!}{2!\cdot(269339-2)!} = 36271613791$ potential synonym pairs<br><br>
Inverted Indices - For each stripe of length 1000 words, there are 1000-Choose-2 combinations of words. Multiplying this by the size of the vocab gives the total number of pairs that would need shuffling prior to reduction. <br>
$Individual Posting Tuple = vocabsize * _{stripe size} C _{choose} = 269339 * _{1000}C_{2} = 269339 * \frac{1000!}{2!*(1000-2)!} = 269339 * 499500 = 134534830500 tuples$

> __c)__ The most frequently used words are mostly 'helper' words, such as articles, pronouns, conjunctions. These are likely not helpful for synonym detection as these figures of speech are sufficiently versatile (as evidenced by their high frequency count!) to connect many different terms regardless of synonym status. 

> __d)__ Of the least frequently used words, most appear to be names, which is not term that is synonomous with another term. These bottom words will likely not yield high quality synonyms.

In [58]:
# part a - write your spark job here 
def EDA1(rdd, n):
    total, top_n, bottom_n = None, None, None
    ############# YOUR CODE HERE ###############
    RDD_5_a = rdd.map(lambda x: x.replace('\t', ' ').lower().split(' ')) \
      .flatMap(lambda x: [(x[i],int(x[5])) for i in range(5)]) \
      .reduceByKey(lambda x,y:x+y)

    total = RDD_5_a.keys().distinct().count() 
    top_n = RDD_5_a.takeOrdered(n,key = lambda x: -x[1])
    bottom_n = RDD_5_a.takeOrdered(n,key = lambda x: x[1])  
    ############# (END) YOUR CODE ##############
    return total, top_n, bottom_n

In [59]:
# part a - run the system test (RUN THIS CELL AS IS... use display cell below to see results)
import time
start = time.time()
vocab_size, most_frequent, least_frequent = EDA1(testRDD, 10)
print("Wall time: {} seconds".format(time.time() - start))
# Expected result on w261_homeworks cluster: Wall time: 1.0887939929962158 seconds

Wall time: 0.577460765838623 seconds

In [60]:
# part a - display results (feel free to modify the formatting code if needed)
print("Vocabulary Size:", vocab_size)
print(" ---- Top Words ----|--- Bottom Words ----")
for (w1, c1), (w2, c2) in zip(most_frequent, least_frequent):
    print(f"{w1:>8} {c1:>10} |{w2:>15} {c2:>3}")

Vocabulary Size: 10
 ---- Top Words ----|--- Bottom Words ----
 was 17 | foolishness 1
 of 17 | best 4
 the 17 | worst 5
 it 16 | wisdom 5
 times 10 | age 8
 age 8 | times 10
 worst 5 | it 16
 wisdom 5 | was 17
 best 4 | of 17
foolishness 1 | the 17

%md Expected output for testRDD:
<pre>
    Vocabulary Size: 10
 ---- Top Words ----|--- Bottom Words ----
     was         17 |    foolishness   1
      of         17 |           best   4
     the         17 |          worst   5
      it         16 |         wisdom   5
   times         10 |            age   8
     age          8 |          times  10
   worst          5 |             it  16
  wisdom          5 |            was  17
    best          4 |             of  17
foolishness       1 |            the  17  
</pre>

In [62]:
# part a - run a single file, ie., a small sample (RUN THIS CELL AS IS)
start = time.time()
vocab_size, most_frequent, least_frequent = EDA1(f1RDD, 10)
print("Wall time: {} seconds".format(time.time() - start))
# Expected result on w261_homeworks cluster: Wall time: 1.9411416053771973 seconds

Wall time: 5.424016237258911 seconds

In [63]:
# part a - display results (feel free to modify the formatting code if needed)
print("Vocabulary Size:", vocab_size)
print(" ---- Top Words ----|--- Bottom Words ----")
for (w1, c1), (w2, c2) in zip(most_frequent, least_frequent):
    print(f"{w1:>8} {c1:>10} |{w2:>15} {c2:>3}")

Vocabulary Size: 36353
 ---- Top Words ----|--- Bottom Words ----
 the 27691943 | jackson's 40
 of 18590950 | nodular 40
 to 11601757 | sicken 40
 in 7470912 | hearse 40
 a 6926743 | florine 40
 and 6150529 | coralie 40
 that 4077421 | japanify 40
 is 4074864 | breastplate 40
 be 3720812 | beaverbrook's 40
 was 2492074 | mansfeld's 40

Expected output for f1RDD
<pre>
Vocabulary Size: 36353
 ---- Top Words ----|--- Bottom Words ----
     the   27691943 |    stakeholder  40
      of   18590950 |          kenny  40
      to   11601757 |         barnes  40
      in    7470912 |         arnall  40
       a    6926743 |     buonaparte  40
     and    6150529 |       puzzling  40
    that    4077421 |             hd  40
      is    4074864 |        corisca  40
      be    3720812 |       cristina  40
     was    2492074 |         durban  40
</pre>

In [65]:
# part a - run full analysis (RUN THIS CELL AS IS)
start = time.time()
vocab_size, most_frequent, least_frequent = EDA1(dataRDD, 10)
print("Wall time: {} seconds".format(time.time() - start))
# Expected result on w261_homeworks cluster: Wall time: 148.80424451828003 seconds

Wall time: 27.09851360321045 seconds

In [66]:
# part a - display results (feel free to modify the formatting code if needed)
print("Vocabulary Size:", vocab_size)
print(" ---- Top Words ----|--- Bottom Words ----")
for (w1, c1), (w2, c2) in zip(most_frequent, least_frequent):
    print(f"{w1:>8} {c1:>10} |{w2:>15} {c2:>3}")

Vocabulary Size: 269339
 ---- Top Words ----|--- Bottom Words ----
 the 5490815394 | parcival 40
 of 3698583299 | schwetzingen 40
 to 2227866570 | cras 40
 in 1421312776 | scholared 40
 a 1361123022 | viscerating 40
 and 1149577477 | scribbler's 40
 that 802921147 | washermen 40
 is 758328796 | rll 40
 be 688707130 | porti 40
 as 492170314 | unmurmuring 40

Expected output for dataRDD:
(bottom words might vary a little due to ties)
<pre>
Vocabulary Size: 269339
 ---- Top Words ----|--- Bottom Words ----
     the 5490815394 |   schwetzingen  40
      of 3698583299 |           cras  40
      to 2227866570 |       parcival  40
      in 1421312776 |          porti  40
       a 1361123022 |    scribbler's  40
     and 1149577477 |      washermen  40
    that  802921147 |    viscerating  40
      is  758328796 |         mildes  40
      be  688707130 |      scholared  40
      as  492170314 |       jaworski  40
</pre>

# Question 6: N-gram EDA part 2 (co-occurrences)

The computational complexity of synonym analysis depends not only on the number of words, but also on the number of co-ocurrences each word has. In this question you'll take a closer look at that aspect of our data. As before, please test each job on small "systems test" (Dickens ngrams) file and on a single file from the Ngram dataset before running the full analysis.

### Q6 Tasks:
* __a) code:__ Write a spark job that computes:
  * the number of unique neighbors (i.e. 5-gram co-occuring words) for each word in the vocabulary. 
  * the top 10 words with the most "neighbors"
  * the bottom 10 words with least "neighbors"
  * a random sample of 1% of the words' neighbor counts  
  __`NOTE:`__ for the last item, please return only the counts and not the words -- we'll go on to use these in a plotting function that expects a list of integers.


* __b) short response:__ Use the provided code to plot a histogram of the sampled list from `a`. Comment on the distribution you observe. How will this distribution affect our synonym detection analysis?

* __c) code + short response:__ Write a Spark Job to compare the top/bottom words from Q5 and from part a. Specifically, what % of the 1000 most/least neighbors words also appear in the list of 1000 most/least frequent words. [__`NOTE:`__ _technically these lists are short enough to comparing in memory on your local machine but please design your Spark job as if we were potentially comparing much larger lists._]

### Q6 Student Answers:

> __b)__ The distribution is heavily skewed towards words with few neighbor words (ie. counts). This inroduces sparsity, which will be a detriment to quality of synonym detection and compute time as compared to a distribution more skewed towards many neighbor words.
<br>
> __c)__ Per script output for dataRDD:<br>
Of the 1000 words with most neighbors, 88.0 percent are also in the list of 1000 most frequent words.<br>
Of the 1000 words with least neighbors, 1.9 percent are also in the list of 1000 least frequent words.<br>

In [70]:
# part a - spark job
def EDA2(rdd,n):
    top_n, bottom_n, sampled_counts = None, None, None
    ############# YOUR CODE HERE ###############
    RDD_6_a = rdd.map(lambda x: x.replace('\t', ' ').lower().split(' ')) \
      .flatMap(lambda x: itertools.combinations(x, 2)) \
      .filter(lambda x: (re.findall('[a-z]+', x[0]))) \
      .filter(lambda x: (re.findall('[a-z]+', x[1]))) \
      .filter(lambda x: (x[0] != x[1])) \
      .map(lambda x: ((x[1], x[0]) if x[1]<x[0] else x)) \
      .distinct() \
      .flatMap(lambda x: [(x[i],int(1)) for i in range(2)]) \
      .reduceByKey(lambda x,y:x+y) \
  
    total = RDD_6_a.keys().distinct().count() 
    top_n = RDD_6_a.takeOrdered(n,key = lambda x: -x[1])
    bottom_n = RDD_6_a.takeOrdered(n,key = lambda x: x[1])  
    one_pct = 0.01*total
    sampled_counts = RDD_6_a.sample(True, one_pct, seed=1) \
      .map(lambda x: x[1]) \
      .collect()
    
    ############# (END) YOUR CODE ##############
    return top_n, bottom_n, sampled_counts

In [71]:
# part a - systems test (RUN THIS CELL AS IS)
start = time.time()
most_nbrs, least_nbrs, sample_counts = EDA2(testRDD, 10)
print("Wall time: {} seconds".format(time.time() - start))
# Expected result on w261_homeworks cluster: Wall time: 2.1029624938964844 seconds

Wall time: 3.5462348461151123 seconds

In [72]:
# part a - display results (feel free to modify the formatting code if needed)
print(" --- Most Co-Words ---|--- Least Co-Words ----")
for (w1, c1), (w2, c2) in zip(most_nbrs, least_nbrs):
    print(f"{w1:>12} {c1:>8} |{w2:>16} {c2:>4}")

--- Most Co-Words ---|--- Least Co-Words ----
 was 9 | foolishness 4
 of 9 | best 5
 the 9 | worst 5
 it 8 | wisdom 5
 times 7 | times 7
 age 7 | age 7
 best 5 | it 8
 worst 5 | was 9
 wisdom 5 | of 9
 foolishness 4 | the 9

Expected output for testRDD:
<pre>
 --- Most Co-Words ---|--- Least Co-Words ----
         was        9 |     foolishness    4
          of        9 |            best    5
         the        9 |           worst    5
          it        8 |          wisdom    5
         age        7 |             age    7
       times        7 |           times    7
        best        5 |              it    8
       worst        5 |             was    9
      wisdom        5 |              of    9
 foolishness        4 |             the    9
 </pre>

In [74]:
# part a - single file test (RUN THIS CELL AS IS)
start = time.time()
most_nbrs, least_nbrs, sample_counts = EDA2(f1RDD, 10)
print("Wall time: {} seconds".format(time.time() - start))
# Expected result on w261_homeworks cluster: Wall time: 7.468951463699341 seconds

Wall time: 28.11562156677246 seconds

In [75]:
# part a - display results (feel free to modify the formatting code if needed)
print(" --- Most Co-Words ---|--- Least Co-Words ----")
for (w1, c1), (w2, c2) in zip(most_nbrs, least_nbrs):
    print(f"{w1:>12} {c1:>8} |{w2:>16} {c2:>4}")

--- Most Co-Words ---|--- Least Co-Words ----
 the 25548 | vo 1
 of 22496 | pizza 2
 and 16489 | premiers 2
 to 14249 | enclaves 2
 in 13891 | selectiveness 2
 a 13045 | trill 2
 that 8011 | noncleaved 2
 is 7947 | dalles 2
 with 7552 | destabilisation 2
 by 7400 | paleography 2

Expected output for f1RDD:
<pre>
 --- Most Co-Words ---|--- Least Co-Words ----
         the    25548 |              vo    1
          of    22496 |      noncleaved    2
         and    16489 |        premiers    2
          to    14249 |        enclaves    2
          in    13891 |   selectiveness    2
           a    13045 |           trill    2
        that     8011 |           pizza    2
          is     7947 |            hoot    2
        with     7552 |     palpitation    2
          by     7400 |            twel    2
</pre>

In [77]:
# part a - full data (RUN THIS CELL AS IS)
start = time.time()
most_nbrs, least_nbrs, sample_counts = EDA2(dataRDD, 10)
print("Wall time: {} seconds".format(time.time() - start))
# Expected result on w261_homeworks cluster: Wall time: 140.08682656288147 seconds

Wall time: 469.1932876110077 seconds

In [78]:
# part a - display results (feel free to modify the formatting code if needed)
print(" --- Most Co-Words ---|--- Least Co-Words ----")
for (w1, c1), (w2, c2) in zip(most_nbrs, least_nbrs):
    print(f"{w1:>12} {c1:>8} |{w2:>16} {c2:>4}")

--- Most Co-Words ---|--- Least Co-Words ----
 the 164982 | cococo 1
 of 155708 | inin 1
 and 132814 | charuhas 1
 in 110615 | ooooooo 1
 to 94358 | iiiii 1
 a 89197 | iiiiii 1
 by 67266 | cnj 1
 with 65127 | choh 1
 that 61174 | neg 1
 as 60652 | cococococo 1

Expected output for dataRDD: 
(bottom words might vary a little due to ties)
<pre>
 --- Most Co-Words ---|--- Least Co-Words ----
         the   164982 |          cococo    1
          of   155708 |            inin    1
         and   132814 |        charuhas    1
          in   110615 |         ooooooo    1
          to    94358 |           iiiii    1
           a    89197 |          iiiiii    1
          by    67266 |             cnj    1
        with    65127 |            choh    1
        that    61174 |             neg    1
          as    60652 |      cococococo    1
</pre>

__`NOTE:`__ _before running the plotting code below, make sure that the variable_ `sample_counts` _points to the list generated in_ `part a`.

In [81]:
# part b - plot histogram (RUN THIS CELL AS IS - feel free to modify format)

# removing extreme upper tail for a better visual
counts = np.array(sample_counts)[np.array(sample_counts) < 6000]
t = sum(np.array(sample_counts) > 6000)
n = len(counts)
print("NOTE: we'll exclude the %s words with more than 6000 nbrs in this %s count sample." % (t,n))

# set up figure
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (15,5))

# plot regular hist
ax1.hist(counts, bins=50)
ax1.set_title('Freqency of Number of Co-Words', color='0.1')
ax1.set_facecolor('0.9')
ax1.tick_params(axis='both', colors='0.1')
ax1.grid(True)

# plot log scale hist
ax2.hist(counts, bins=50)
ax2.set_title('(log)Freqency of Number of Co-Words', color='0.1')
ax2.set_facecolor('0.9')
ax2.tick_params(axis='both', colors='0.1')
ax2.grid(True)
plt.yscale('log')


NOTE: we'll exclude the 5351 words with more than 6000 nbrs in this 13213171 count sample.

In [82]:
display(fig)

In [83]:
# part c - spark job
def compareRankings(rdd1, rdd2):
    percent_overlap = None
    ############# YOUR CODE HERE ###############
    #rdd1 q5 is frequency
    #rdd2 q6 is neighbors
    #seeking % of rdd2 words (distinct, not freq) that are in rdd1 --compare inner join to rdd2 wc
    rdd2_in_rdd1 = rdd2.join(rdd1) \
      .map(lambda x: x[0]) \
      .distinct().count()
    rdd2_wc = rdd2.keys().count()
    percent_overlap = 100* rdd2_in_rdd1 / rdd2_wc
    ############# (END) YOUR CODE ##############
    return percent_overlap

In [84]:
# part c - get lists for comparison (RUN THIS CELL AS IS...)
# (... then change 'testRDD' to 'f1RDD'/'dataRDD' when ready)
total, topWords, bottomWords = EDA1(dataRDD, 1000)
topNbrs, bottomNbrs, sample_counts = EDA2(dataRDD, 1000)
twRDD = sc.parallelize(topWords)
bwRDD = sc.parallelize(bottomWords)
tnRDD = sc.parallelize(topNbrs)
bnRDD = sc.parallelize(bottomNbrs)
top_overlap = compareRankings(tnRDD, twRDD)
bottom_overlap = compareRankings(bnRDD,bwRDD)
print(f"Of the 1000 words with most neighbors, {top_overlap} percent are also in the list of 1000 most frequent words.")
print(f"Of the 1000 words with least neighbors, {bottom_overlap} percent are also in the list of 1000 least frequent words.")

Of the 1000 words with most neighbors, 88.0 percent are also in the list of 1000 most frequent words.
Of the 1000 words with least neighbors, 1.9 percent are also in the list of 1000 least frequent words.

# Question 7: Basis Vocabulary & Stripes

Every word that appears in our data is a potential feature for our synonym detection analysis. However as we've discussed, some are likely to be more useful than others. In this question, you'll choose a judicious subset of these words to form our 'basis vocabulary' (i.e. feature set). Practically speaking, this means that when we build our stripes, we are only going to keep track of when a term co-occurs with one of these basis words. 


### Q7 Tasks:
* __a) short response:__ Suppose we were deciding between two different basis vocabularies: the 1000 most frequent words or the 1000 least frequent words. How would this choice impact the quality of the synonyms we are able to detect? How does this choice relate to the ideas of 'overfitting' or 'underfitting' a training set?

* __b) short response:__ If we had a much larger dataset, computing the full ordered list of words would be extremely expensive. If we need to none-the-less get an estimate of word frequency in order to decide on a basis vocabulary (feature set), what alternative strategy could we take?

* __c) code:__ Write a spark job that does the following:
  * tokenizes, removes stopwords and computes a word count on the ngram data
  * subsets the top 10,000 words (these are the terms we'll consider as potential synonyms)
  * subsets words 9,000-9,999 (this will be our 1,000 word basis vocabulary)    
  (to put it another way - of the top 10,000 words, the bottom 1,000 form the basis vocabulary)
  * saves the full 10K word list and the 1K basis vocabulary to file for use in `d`.  
  
  __NOTE:__ _to ensure consistency in results please use only the provided list of stopwords._  
  __NOTE:__ _as always, be sure to test your code on small files as you develop it._  

* __d) code:__ Write a spark job that builds co-occurrence stripes for the top 10K words in the ngram data using the basis vocabulary you developed in `part c`. This job/function, unlike others so far, should return an RDD (which we will then use in q8).

### Q7 Student Answers:
> __a)__ <br>
1000 most frequent/Underfit - Most frequent words could be most frequent because they are overly generic terms that are a superset of terms where there are synonyms and antonyms. For exampple, in `testRDD`, `times` is more frequent than `best` or `worst` and they appear in the same n-grams. Yet, `best` and `worst` have opposite meanings and are descriptors of `times`. This generality creates a high level of bias which lends towards an UNDERFIT scenario.<br>   
1000 least frequent/Overfit - Least frequent words could be least frequent because they only applicable are for very specific usage cases where there are not many relevent synonyms. This could create an OVERFIT scenario as the trained model fails to generalize the larger test population.<br><br>

> __b)__ To reduce the computing load (ie. expense) of a large dataset, a smaller, representative dataset could be used. This smaller dataset could be obtained by sampling the larger dataset.

In [87]:
# part c - provided stopwords (RUN THIS CELL AS IS)
STOPWORDS =  ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
              'ourselves', 'you', 'your', 'yours', 'yourself', 
              'yourselves', 'he', 'him', 'his', 'himself', 'she', 
              'her', 'hers', 'herself', 'it', 'its', 'itself', 
              'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', 
              'these', 'those', 'am', 'is', 'are', 'was', 'were', 
              'be', 'been', 'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
              'but', 'if', 'or', 'because', 'as', 'until', 'while', 
              'of', 'at', 'by', 'for', 'with', 'about', 'against', 
              'between', 'into', 'through', 'during', 'before', 
              'after', 'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 'again', 
              'further', 'then', 'once', 'here', 'there', 'when', 
              'where', 'why', 'how', 'all', 'any', 'both', 'each', 
              'few', 'more', 'most', 'other', 'some', 'such', 'no', 
              'nor', 'not', 'only', 'own', 'same', 'so', 'than', 
              'too', 'very', 'should', 'can', 'now', 'will', 'just', 
              'would', 'could', 'may', 'must', 'one', 'much', "it's",
              "can't", "won't", "don't", "shouldn't", "hasn't"]

In [88]:
# part c - spark job
def get_vocab(rdd, n_total, n_basis):
    vocab, basis = None, None
    ############# YOUR CODE HERE ###############
    vocab = rdd.map(lambda x: x.replace('\t', ' ').lower().split(' ')) \
      .flatMap(lambda x: [(x[i],int(x[5])) for i in range(5)]) \
      .reduceByKey(lambda x,y:x+y) \
      .filter(lambda x: x[0] not in STOPWORDS) \
      .sortBy(lambda x: x[1],ascending = False) \
      .map(lambda x: x[0]) \
      .take(n_total)

    basis = vocab[-1*n_basis:]  
    ############# (END) YOUR CODE ##############
    return vocab, basis

In [89]:
# part c - run your job (RUN THIS CELL AS IS)
start = time.time()
VOCAB, BASIS = get_vocab(dataRDD, 10000, 1000)
print("Wall time: {} seconds".format(time.time() - start))
# Expected wall time on w261_homeworks cluster: 33.64917516708374 seconds

Wall time: 37.555487871170044 seconds

In [90]:
dbutils.fs.put(hw3_path+"vocabulary.txt",str(VOCAB),True)
dbutils.fs.put(hw3_path+"basis.txt",str(BASIS),True)

Wrote 113523 bytes.
Wrote 11948 bytes.
Out[252]: True

* __d) code:__ Write a spark job that builds co-occurrence stripes for the top 10K words in the ngram data using the basis vocabulary you developed in `part c`. This job/function, unlike others so far, should return an RDD (which we will then use in q8).

In [92]:
# part d - spark job
def buildStripes(rdd, vocab, basis):
    stripesRDD = None
    ############# YOUR CODE HERE ###############
    #Establishing broadcast variables for vocab & basis. This is to maintain the variable in the cache of each executor which minimizes network traffic each time the variable is needed.
    vocab_bcast = sc.broadcast(vocab)
    basis_bcast = sc.broadcast(basis)
    
    stripesRDD = rdd.map(lambda line: line.lower().split("\t")[0].split(" ")) \
               .flatMap(lambda x: itertools.permutations(x,2)) \
               .filter(lambda x: x[0] in vocab_bcast.value) \
               .filter(lambda x: x[1] in basis_bcast.value) \
               .groupByKey() \
               .mapValues(lambda x: set(x)) 
#     stripesRDD
#                .flatMap(lambda x: permutations(x)) \          
    ############# (END) YOUR CODE ##############
    return stripesRDD

In [93]:
# part d - run your systems test (RUN THIS CELL AS IS)
VOCAB, BASIS = get_vocab(testRDD, 10, 10)
testStripesRDD = buildStripes(testRDD, VOCAB, BASIS)
start = time.time()
print(testStripesRDD.collect())
print("Wall time: {} seconds".format(time.time() - start))
# Expected wall time on w261_homeworks cluster: 0.11356496810913086 seconds
# Expected results
'''
[('worst', {'times'}), ('best', {'times'}), ('foolishness', {'age'}), ('times', {'age', 'best', 'worst'}), ('age', {'wisdom', 'foolishness', 'times'}), ('wisdom', {'age'})]
'''

[('worst', {'times'}), ('best', {'times'}), ('foolishness', {'age'}), ('times', {'age', 'best', 'worst'}), ('age', {'wisdom', 'foolishness', 'times'}), ('wisdom', {'age'})]
Wall time: 0.18471550941467285 seconds
Out[362]: "\n[('worst', {'times'}), ('best', {'times'}), ('foolishness', {'age'}), ('times', {'age', 'best', 'worst'}), ('age', {'wisdom', 'foolishness', 'times'}), ('wisdom', {'age'})]\n"

In [94]:
# part d - run your single file test (RUN THIS CELL AS IS)
VOCAB, BASIS = get_vocab(f1RDD, 10000, 1000)
f1StripesRDD = buildStripes(f1RDD, VOCAB, BASIS).cache()
start = time.time()
print(f1StripesRDD.top(5))
print("Wall time: {} seconds".format(time.time() - start))
# Expected wall time on w261_homeworks cluster: 1.3129394054412842 seconds
# Expected results
'''
[('zur', {'zur'}), ('zippor', {'balak'}), ('zedong', {'mao'}), ('zeal', {'infallibility'}), 
('youth', {'mould', 'constrained'})]
'''

[('zur', {'zur'}), ('zippor', {'balak'}), ('zedong', {'mao'}), ('zeal', {'infallibility'}), ('youth', {'mould', 'constrained'})]
Wall time: 327.1786403656006 seconds
Out[363]: "\n[('zur', {'zur'}), ('zippor', {'balak'}), ('zedong', {'mao'}), ('zeal', {'infallibility'}), \n('youth', {'mould', 'constrained'})]\n"

In [95]:
# part d - run the full analysis and take a look at a few stripes (RUN THIS CELL AS IS)
VOCAB = ast.literal_eval(open(hw3_path_open+"vocabulary.txt", "r").read())
BASIS = ast.literal_eval(open(hw3_path_open+"basis.txt", "r").read())
stripesRDD = buildStripes(dataRDD, VOCAB, BASIS).cache()

start = time.time()
for wrd, stripe in stripesRDD.top(3):
    print(wrd)
    print(list(stripe))
    print('-------')
print("Wall time: {} seconds".format(time.time() - start))
# Expected Wall time on w261_homeworks cluster:  25.928858757019043 seconds
# Expected results
'''
zones
['remotest', 'adhesion', 'buffer', 'subdivided', 'environments', 'gaza', 'saturation', 'localities', 'uppermost', 'warmer', 'residential', 'parks']
-------
zone
['tribal', 'narrower', 'fibrous', 'saturation', 'originate', 'auxiliary', 'ie', 'buffer', 'transitional', 'turbulent', 'vomiting', 'americas', 'articular', 'poorly', 'intervening', 'officially', 'accumulate', 'assisting', 'flexor', 'traversed', 'uppermost', 'unusually', 'cartilage', 'inorganic', 'illuminated', 'glowing', 'contamination', 'trigger', 'defines', 'masculine', 'avoidance', 'cracks', 'southeastern', 'penis', 'residential', 'atlas', 'excitation', 'persia', 'diffuse', 'subdivided', 'alaska', 'guides', 'au', 'sandy', 'penetrating', 'parked']
-------
zinc
['ammonium', 'coating', 'pancreas', 'insoluble', "alzheimer's", 'diamond', 'radioactive', 'metallic', 'weighing', 'dysfunction', 'wasting', 'phosphorus', 'transcription', 'dipped', 'hydroxide', 'burns', 'leukemia', 'dietary']
-------
'''

zones
['remotest', 'adhesion', 'residential', 'subdivided', 'environments', 'gaza', 'saturation', 'localities', 'uppermost', 'warmer', 'buffer', 'parks']
-------
zone
['sandy', 'tribal', 'narrower', 'fibrous', 'saturation', 'originate', 'auxiliary', 'ie', 'buffer', 'transitional', 'turbulent', 'vomiting', 'articular', 'poorly', 'intervening', 'officially', 'accumulate', 'assisting', 'flexor', 'traversed', 'uppermost', 'unusually', 'cartilage', 'inorganic', 'illuminated', 'glowing', 'contamination', 'trigger', 'masculine', 'defines', 'avoidance', 'residential', 'cracks', 'penis', 'southeastern', 'atlas', 'excitation', 'persia', 'diffuse', 'subdivided', 'alaska', 'guides', 'au', 'americas', 'penetrating', 'parked']
-------
zinc
['ammonium', 'coating', 'pancreas', 'insoluble', 'radioactive', "alzheimer's", 'diamond', 'metallic', 'weighing', 'dysfunction', 'wasting', 'phosphorus', 'transcription', 'dipped', 'hydroxide', 'burns', 'leukemia', 'dietary']
-------
Wall time: 4185.506931781769 seconds
Out[364]: '\nzones\n[\'remotest\', \'adhesion\', \'buffer\', \'subdivided\', \'environments\', \'gaza\', \'saturation\', \'localities\', \'uppermost\', \'warmer\', \'residential\', \'parks\']\n-------\nzone\n[\'tribal\', \'narrower\', \'fibrous\', \'saturation\', \'originate\', \'auxiliary\', \'ie\', \'buffer\', \'transitional\', \'turbulent\', \'vomiting\', \'americas\', \'articular\', \'poorly\', \'intervening\', \'officially\', \'accumulate\', \'assisting\', \'flexor\', \'traversed\', \'uppermost\', \'unusually\', \'cartilage\', \'inorganic\', \'illuminated\', \'glowing\', \'contamination\', \'trigger\', \'defines\', \'masculine\', \'avoidance\', \'cracks\', \'southeastern\', \'penis\', \'residential\', \'atlas\', \'excitation\', \'persia\', \'diffuse\', \'subdivided\', \'alaska\', \'guides\', \'au\', \'sandy\', \'penetrating\', \'parked\']\n-------\nzinc\n[\'ammonium\', \'coating\', \'pancreas\', \'insoluble\', "alzheimer\'s", \'diamond\', \'radioactive\', \'metallic\', \'weighing\', \'dysfunction\', \'wasting\', \'phosphorus\', \'transcription\', \'dipped\', \'hydroxide\', \'burns\', \'leukemia\', \'dietary\']\n-------\n'

In [96]:
# part d - save your full stripes to file for ease of retrival later... (OPTIONAL)
stripesRDD.saveAsTextFile(hw3_path+'stripes')

# Question 8: Synonym Detection

We're now ready to perform the main synonym detection analysis. In the tasks below you will compute cosine, jaccard, dice and overlap similarity measurements for each pair of words in our vocabulary and then sort your results to find the most similar pairs of words in this dataset. __`IMPORTANT:`__ When you get to the sorting step please __sort on cosine similarity__ only, so that we can ensure consistent results from student to student. 

Remember to test each step of your work with the small files before running your code on the full dataset. This is a computationally intense task: well designed code can be the difference between a 20min job and a 2hr job. __`NOTE:`__ _as you are designing your code you may want to review questions 3 and 4 where we modeled some of the key pieces of this analysis._

### Q8 Tasks:
* __a) short response:__ In question 7 you wrote a function that would create word stripes for each `term` in our vocabulary. These word stripes are essentially an 'embedded representation' of the `term`'s meaning. What is the 'feature space' for this representation? (i.e. what are the features of our 1-hot encoded vectors?). What is the maximum length of a stripe?

* __b) short response:__ Remember that we are going to treat these stripes as 'documents' and perform similarity analysis on them. The first step is to emit postings which then get collected to form an 'inverted index.' How many rows will there be in our inverted index? Explain.

* __c) short response:__ In the demo from question 2, we were able to compute the cosine similarity directly from the stripes (we did this using their vector form, but could have used the list instead). So why do we need the inverted index? (__`HINT:`__ _see your answer to Q4a & Q4b_)

* __d) code:__ Write a spark job that does the following:
  * loops over the stripes from Q7 and emits postings for the `term` (_remember stripe = document_)
  * aggregates the postings to create an inverted index
  * loops over all pairs of `term`s that appear in the same inverted index and emits co-occurrence counts
  * aggregates co-occurrences
  * uses the counts (along with the accompanying information) to compute the cosine, jacard, dice and overlap similarity metrics for each pair of words in the vocabulary 
  * retrieve the top 20 and bottom 20 most/least similar pairs of words
  * also returned the cached sorted RDD for use in the next question  
  __`NOTE 1`:__ _Don't forget to include the stripe length when you are creating the postings & co-occurrence pairs. A composite key is the way to go here._  
  __`NOTE 2`:__ _Please make sure that your final results are sorted according to cosine similarity otherwise your results may not match the expected result & you will be marked wrong._
  
* __e) code:__ Comment on the quality of the "synonyms" your analysis comes up with. Do you notice anything odd about these pairs of words? Discuss at least one idea for how you might go about improving on the analysis.

### Q8 Student Answers:
> __a)__ The feature space for the 'word stripe' representation of synonyms is the entirety of the 1000-term `basis` vocabulary set. The maximum stripe length is 1000, corresponding to a `vocab` term that has shared an ngram with each of the words in the `basis` data set.

> __b)__ There will be a maximum of 1000 rows in our inverted index, with a row per `basis` term. The number could be less than 1000 if basis terms did not co-occur in n-grams with other `vocab` terms.

> __c)__ The inverted index is an aggregation that reduces the computing overhead as compared to a sparse one-hot encoded matrix (many 0s in memory).

> __e)__ <br>
Quality - From a the perspective of a person (fairly developed speech abilities), the quality of synonyms is rather poor, regardless of high similarity scores.<br>
Anything odd - Synonyms shown are not the same figures-of-speech, nor interchangeble in sentences.<br>
Improving - Categorizing terms by figure-of speech and performing similarity analyses filtered on similar figures-of-speech should improve results from a human perspecitve.

In [99]:
# helper function for pretty printing (RUN THIS CELL AS IS)
def displayOutput(lines):
    template = "{:25}|{:6}, {:7}, {:7}, {:5}"
    print(template.format("Pair", "Cosine", "Jaccard", "Overlap", "Dice"))
    for pair, scores in lines:
        scores = [round(s,4) for s in scores]
        print(template.format(pair, *scores))

__`TIP:`__ Feel free to define helper functions within the main function to help you organize your code. Readability is important! Eg:
```
def similarityAnlysis(stripesRDD):
    """main docstring"""
    
    simScoresRDD, top_n, bottom_n = None, None, None
    
    ############ YOUR CODE HERE ###########
    def helper1():
        """helper docstring"""
        return x
        
    def helper2():
        """helper docstring"""
        return x
        
    # main spark job starts here
    
        ...etc
    ############ (END) YOUR CODE ###########
    return simScoresRDD, top_n, bottom_n
```

In [101]:
# part d - write your spark job in the space provided
def similarityAnalysis(stripesRDD, n):
    """
    This function defines a Spark DAG to compute cosine, jaccard, 
    overlap and dice scores for each pair of words in the stripes
    provided. 
    
    Output: an RDD, a list of top n, a list of bottom n
    """
    simScoresRDD, top_n, bottom_n = None, None, None
    
    ############### YOUR CODE HERE ################
    def splitWords_q8(pair):
        """Mapper 2: tokenize each document and emit postings."""
        doc, text = pair
    #         words = text.split(" ")
        for w in text:
            yield (w, [(doc,len(text))])

    def makeCompositeKey_q8(inverted_index):
        """Mapper 3: loop over postings and yield pairs."""
        word, postings = inverted_index
        # taking advantage of symmetry, output only (a,b), but not (b,a)
        for subset in itertools.combinations(sorted(postings), 2):
            yield (str(subset), 1) 

    def cal_sim_q8(line):
        """Mapper 4: compute similarity scores"""
        (doc1, n1), (doc2, n2) = ast.literal_eval(line[0])
        total = int(line[1])
        jaccard = total / float(int(n1) + int(n2) - total)
        cosine = total / float((int(n1)**0.5) * int(n1)**0.5)
        dice = 2 * total / float(int(n1) + int(n2))
        overlap = total / float(min(int(n1), int(n2)))
        yield (doc1+" - "+doc2, (cosine, jaccard, overlap, dice))


    result = stripesRDD.flatMap(splitWords_q8) \
                 .reduceByKey(lambda x,y : x+y) \
                 .flatMap(makeCompositeKey_q8) \
                 .reduceByKey(lambda x,y : x+y) \
                 .flatMap(cal_sim_q8) \
                 .cache()

    top_n = result.takeOrdered(n, key=lambda x: -x[1][0])
    bottom_n = result.takeOrdered(n, key=lambda x: x[1][0])    

    ############### (END) YOUR CODE ##############
    return result, top_n, bottom_n

In [102]:
# part d - run the system test (RUN THIS CELL AS IS... use display cell below to see results)
start = time.time()
testResult, top_n, bottom_n = similarityAnalysis(testStripesRDD, 10)
print("Wall time: {} seconds".format(time.time() - start))
# Expected result on w261_homeworks cluster: Wall time: 0.5179581642150879 seconds

Wall time: 38.02394127845764 seconds

In [103]:
# part d - run the system test (RUN THIS CELL AS IS... use display cell below to see results)
start = time.time()
f1Result, top_n, bottom_n = similarityAnalysis(f1StripesRDD, 10)
print("Wall time: {} seconds".format(time.time() - start))
# Expected result  on w261_homeworks cluster:  Wall time: 1.066291332244873 seconds


Wall time: 457.01783204078674 seconds

In [104]:
# part d - run the system test (RUN THIS CELL AS IS... use display cell below to see results)
start = time.time()
result, top_n, bottom_n = similarityAnalysis(stripesRDD, 20)
print("Wall time: {} seconds".format(time.time() - start))
# Expected result on w261_homeworks cluster: Wall time: 138.1922881603241 seconds

Wall time: 3530.7551140785217 seconds

# Note to Grader 
Hi Grader. I note that the above job on the full data set takes way too long (50x expected!!). I think it may be that the stripesRDD was calculated (and cached) not within the day, so the cache copy was discarded. Or it may be that 3 other people are running jobs on the cluster! I noted that some stages were skipped by Spark. This may be interfering with results. The results were not as expected. Interestingly, I had previously calc'd for f1StripesRDD and MATCHED expected output (for least similar, ...most similar were 1.0 scores, albeit different pairs), and now I do not. The fact that results are reduced cosine similarirty indicates missing data. Results below are set on f1StripesRDD.

In [106]:
displayOutput(top_n)

Pair |Cosine, Jaccard, Overlap, Dice 
judicium - per | 1.0, 0.2727, 1.0, 0.4286
legale - per | 1.0, 0.2727, 1.0, 0.4286
parium - per | 1.0, 0.2727, 1.0, 0.4286
ease - great | 1.0, 0.04, 1.0, 0.0769
ease - manual | 1.0, 0.3333, 1.0, 0.5
ease - power | 1.0, 0.0625, 1.0, 0.1176
ease - ship | 1.0, 0.3333, 1.0, 0.5
ease - show | 1.0, 0.25, 1.0, 0.4
ease - wholly | 1.0, 0.3333, 1.0, 0.5
incentives - lend | 1.0, 1.0, 1.0, 1.0

In [107]:
displayOutput(bottom_n)

Pair |Cosine, Jaccard, Overlap, Dice 
time - usually |0.0286, 0.025, 0.1667, 0.0488
time - towers |0.0286, 0.0286, 1.0, 0.0556
time - tumor |0.0286, 0.0286, 1.0, 0.0556
time - used |0.0286, 0.0192, 0.0556, 0.0377
time - whole |0.0286, 0.0208, 0.0714, 0.0408
time - true |0.0286, 0.0263, 0.25, 0.0513
time - yet |0.0286, 0.0222, 0.0909, 0.0435
time - wounds |0.0286, 0.0286, 1.0, 0.0556
time - wisdom |0.0286, 0.027, 0.3333, 0.0526
time - upon |0.0286, 0.0159, 0.0345, 0.0312

__Expected output f1RDD:__  
<table>
<th>MOST SIMILAR:</th>
<th>LEAST SIMILAR:</th>
<tr><td><pre>
Pair                     |Cosine, Jaccard, Overlap, Dice 
commentary - lady        |   1.0,     1.0,     1.0,   1.0
commentary - toes        |   1.0,     1.0,     1.0,   1.0
commentary - reply       |   1.0,     1.0,     1.0,   1.0
curious - tone           |   1.0,     1.0,     1.0,   1.0
curious - lady           |   1.0,     1.0,     1.0,   1.0
curious - owe            |   1.0,     1.0,     1.0,   1.0
lady - tone              |   1.0,     1.0,     1.0,   1.0
reply - tone             |   1.0,     1.0,     1.0,   1.0
lady - toes              |   1.0,     1.0,     1.0,   1.0
lady - reply             |   1.0,     1.0,     1.0,   1.0
</pre></td>
<td><pre>

Pair                     |Cosine, Jaccard, Overlap, Dice 
part - time              |0.0294,  0.0149,  0.0303, 0.0294
time - upon              |0.0314,  0.0159,  0.0345, 0.0312
time - two               |0.0314,  0.0159,  0.0345, 0.0312
made - time              |0.0325,  0.0164,   0.037, 0.0323
first - time             |0.0338,  0.0169,    0.04, 0.0333
new - time               |0.0352,  0.0175,  0.0435, 0.0345
part - us                |0.0355,  0.0179,  0.0417, 0.0351
little - part            |0.0355,  0.0179,  0.0417, 0.0351
made - two               |0.0357,  0.0182,   0.037, 0.0357
made - upon              |0.0357,  0.0182,   0.037, 0.0357
</pre></td></tr>
</table>

__Expected output dataRDD:__  
<table>
<th>Most Similar</th>
<th>Least Similar</th>
<tr><td><pre>
Pair           |Cosine, Jaccard, Overlap, Dice 
first - time   |  0.89,  0.8012,  0.9149, 0.8897
time - well    |0.8895,   0.801,   0.892, 0.8895
great - time   | 0.875,  0.7757,   0.925, 0.8737
part - well    | 0.874,  0.7755,  0.9018, 0.8735
first - well   |0.8717,  0.7722,  0.8936, 0.8715
part - time    |0.8715,  0.7715,  0.9018, 0.871
time - upon    |0.8668,   0.763,  0.9152, 0.8656
made - time    | 0.866,  0.7619,  0.9109, 0.8649
made - well    |0.8601,  0.7531,  0.9022, 0.8592
time - way     |0.8587,  0.7487,  0.9259, 0.8563
great - well   |0.8526,  0.7412,  0.8988, 0.8514
time - two     |0.8517,  0.7389,  0.9094, 0.8498
first - great  |0.8497,  0.7381,  0.8738, 0.8493
first - part   |0.8471,  0.7348,  0.8527, 0.8471
great - upon   |0.8464,  0.7338,  0.8475, 0.8464
upon - well    |0.8444,   0.729,   0.889, 0.8433
new - time     |0.8426,   0.724,  0.9133, 0.8399
first - two    |0.8411,  0.7249,  0.8737, 0.8405
way - well     |0.8357,  0.7146,  0.8986, 0.8335
time - us      |0.8357,  0.7105,  0.9318, 0.8308
</pre></td>
<td><pre>
Pair                  |Cosine, Jaccard, Overlap, Dice 
region - write        |0.0067,  0.0032,  0.0085, 0.0065
relation - snow       |0.0067,  0.0026,  0.0141, 0.0052
cardiac - took        |0.0074,  0.0023,  0.0217, 0.0045
ever - tumor          |0.0076,   0.002,  0.0263, 0.004
came - tumor          |0.0076,   0.002,  0.0263, 0.004
let - therapy         |0.0076,   0.003,  0.0161, 0.0059
related - stay        |0.0078,  0.0036,  0.0116, 0.0072
factors - hear        |0.0078,  0.0039,  0.0094, 0.0077
implications - round  |0.0078,  0.0033,  0.0145, 0.0066
came - proteins       |0.0079,   0.002,  0.0286, 0.0041
population - window   |0.0079,  0.0039,    0.01, 0.0077
love - proportional   | 0.008,  0.0029,  0.0185, 0.0058
got - multiple        | 0.008,  0.0034,  0.0149, 0.0067
changes - fort        |0.0081,  0.0032,  0.0161, 0.0065
layer - wife          |0.0081,  0.0038,  0.0119, 0.0075
five - sympathy       |0.0081,  0.0034,  0.0149, 0.0068
arrival - essential   |0.0081,   0.004,  0.0093, 0.008
desert - function     |0.0081,  0.0031,  0.0175, 0.0062
fundamental - stood   |0.0081,  0.0038,  0.0115, 0.0077
patients - plain      |0.0081,   0.004,  0.0103, 0.0079
</pre></td></tr>
</table>

### Congratulations, you have completed HW3! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform